In [27]:
from importlib import reload
from functools import reduce

def reload_all():
    import tsm; reload(tsm)
    import _keys; reload(_keys)
    import blizzard; reload(blizzard)
    import config; reload(config)
    import kvstore; reload(kvstore)
    import snapshot; reload(snapshot)
    import elemental_arbitrage; reload(elemental_arbitrage)
    import bliz_tsm_join; reload(bliz_tsm_join)
    import crafting; reload(crafting)
    import kvstore; reload(kvstore)
    import crafting; reload(crafting)
    import procurement; reload(procurement)
reload_all()
from itables import show

import pandas as pd


def without_columns(columns, df):
    return reduce(lambda acc, x: acc.drop(x, axis=1), columns, df)

In [28]:
from snapshot import SnapshotProcessor
from tsm import auction_house_snapshot
from config import tsm_region_id
from config import tsm_realm_id
from config import tsm_ah_id
from config import tsm_cache_dir

def tsm_ah_snapper():
    return auction_house_snapshot(tsm_region_id, tsm_realm_id, tsm_ah_id)

tsm_ah_snap = SnapshotProcessor(tsm_ah_snapper, cache_dir=tsm_cache_dir)
tsm_ah = tsm_ah_snap.get(max_age_seconds=3000)

In [29]:
from snapshot import SnapshotProcessor
from blizzard import auction_data
from blizzard import auction_summary
from config import blizzard_realm_id
from config import blizzard_ah_id
from config import blizzard_cache_dir

def bliz_ah_snapper():
    return auction_data(blizzard_realm_id, blizzard_ah_id)

bliz_ah_snap = SnapshotProcessor(bliz_ah_snapper, cache_dir=blizzard_cache_dir)
bliz_ah = bliz_ah_snap.get(max_age_seconds=3000)

In [30]:
from kvstore import InefficientKVStore
from blizzard import ItemLookup
from blizzard import collapse_languages
from config import blizzard_item_cache
from config import blizzard_item_reverse_cache

items = ItemLookup(
    InefficientKVStore(blizzard_item_cache),
    InefficientKVStore(blizzard_item_reverse_cache),
)

In [31]:
from bliz_tsm_join import ItemInfoAggregator

iii = ItemInfoAggregator(items, bliz_ah, tsm_ah, InefficientKVStore("aggregator.pkl"))

In [32]:
from functools import partial

from cytoolz import groupby
from cytoolz import topk

from crafting import Craft
from crafting import Procure
from crafting import Recipes
from crafting import coalesce
from crafting import SpecificProcure
from combined import dnf


r = Recipes(items)
i = r.ingredients


with open("recipes.txt") as f:
    r.read_from_file(f)

# TSM Ledger

In [1]:
from cytoolz import groupby
import pandas as pd

server = "Eranikus"
modes = ["sales", "purchases", "expired", "canceled"]

all_dfs = []
for mode in modes:
    try:
        mode_df = pd.read_csv(f"Accounting_{server}_{mode}.csv")
        mode_df["mode"] = mode
        all_dfs.append(mode_df)
    except FileNotFoundError:
        continue

df_all = pd.concat(all_dfs)

def summarize_tsm_by_mask(mask):
    df = df_all[mask]
    df["total_value"] = df["price"] * df["quantity"]
    by_item = df.groupby("itemName")
    stack_values = by_item["total_value"].sum()
    quantities = by_item["quantity"].sum()
    prices = by_item["price"].mean()
    prices.name = "unit price"
    prices_volatility = by_item["price"].std()
    prices_volatility.name = "price volatility"
    prices_volatility_pct = prices_volatility
    return pd.concat([stack_values/1e4, quantities, prices/1e4, prices_volatility/1e4], axis=1)

In [2]:
purchases = df_all[df_all["mode"] == "purchases"]
purchases["weightPrice"] = purchases["price"]*purchases["quantity"]
purchases.groupby("itemName")["weightPrice"].sum() / purchases.groupby("itemName")["quantity"].sum()
purchases.groupby("itemName")["price"].max()

<ipython-input-2-67a73a8646a3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchases["weightPrice"] = purchases["price"]*purchases["quantity"]


itemName
?                                   600.0
Adder's Tongue                    13670.0
Arcane Crystal                    12498.0
Arcanum of Burning Mysteries    1200000.0
Autumn's Glow                     66078.0
                                  ...    
Thorium Bar                       15000.0
Thorium Ore                        9998.0
Truesilver Ore                    34898.0
Twilight Opal                     40736.0
Wild Spineleaf                     4250.0
Name: price, Length: 93, dtype: float64

In [25]:
mq = df_all.groupby(["itemString", "mode"])["quantity"].sum().reset_index()

mq_sales = mq[mq["mode"] == "sales"].drop(columns=["mode"]).set_index("itemString")
mq_expired = mq[mq["mode"] == "expired"].drop(columns=["mode"]).set_index("itemString")

(mq_sales["quantity"] / (mq_sales["quantity"] + mq_expired["quantity"])).dropna().sort_values()

itemString
i:44958    0.049628
i:13463    0.053571
i:43571    0.055085
i:6370     0.073801
i:46378    0.082164
             ...   
i:33447    0.805556
i:33448    0.861111
i:11176    0.869565
i:3577     0.890909
i:42942    0.969697
Name: quantity, Length: 81, dtype: float64

In [34]:
df_all[df_all["itemName"] == "Baked Manta Ray"]

,itemString,itemName,stackSize,quantity,price,otherPlayer,player,time,source,mode
2,i:42942,Baked Manta Ray,10,10,10218.0,Selera,Beet,1675140058,Auction,sales
10,i:42942,Baked Manta Ray,10,30,10218.0,Ayobigdog,Beet,1675144977,Auction,sales
11,i:42942,Baked Manta Ray,4,4,10218.0,Ayobigdog,Beet,1675144988,Auction,sales
253,i:42942,Baked Manta Ray,4,4,8547.0,Flodena,Beet,1675875144,Auction,sales
424,i:42942,Baked Manta Ray,10,40,8843.0,Tanukiuwu,Beet,1676488984,Auction,sales
443,i:42942,Baked Manta Ray,10,20,9126.0,Azurawux,Beet,1676582781,Auction,sales
449,i:42942,Baked Manta Ray,10,20,9126.0,Proteinbar,Beet,1676589622,Auction,sales
134,i:42942,Baked Manta Ray,4,4,NaN,NaN,Beet,1675857952,NaN,expired


# Item Reference

In [42]:
with open("notable_items.txt") as f:
    _names = [line.strip() for line in f]

desired_names = list(set(_names))
desired_ids = [items.get_id(name) for name in desired_names if not name.startswith("#")]

## Tabular Reference

In [43]:
df_ref = pd.DataFrame(iii.get(item_name=i) for i in _names)

In [12]:
import pickle
from cytoolz import get as pick_from_dict
from cytoolz import sliding_window
import glob
import itertools
from blizzard import auction_summary
import re
import numpy as np
import datetime
np.set_printoptions(suppress=True)

date_re = re.compile(r"202.-..-..T..-..-..")
def _timestamp_from_name(name):
    cap = date_re.search(name).group(0)
    return datetime.datetime.strptime(cap, "%Y-%m-%dT%H-%M-%S").timestamp()


by_item = {}
timeseries = []

for snapshot in sorted(glob.glob("bliz-ah/*")):
    with open(snapshot, "rb") as f:
        entries = pickle.load(f).values()
        timeseries.extend(
            [
                {"timestamp": _timestamp_from_name(snapshot), "item_name": items.get_name(item_id), **auction_summary(item_data)}
                for (item_id, item_data) in zip(desired_ids, entries)
            ]
        )
        for (item_id, item_data) in zip(desired_ids, entries):
            name = items.get_name(item_id)
            by_item[name] = by_item.get(name, []) + [{"timestamp": date_re.search(snapshot).group(0), "item_name": items.get_name(item_id), **auction_summary(item_data)}]

def timeseries_of(key, default=np.nan):
    def _timeseries_of(item_name):
        return np.array([v.get(key, default) for v in by_item[item_name]])
    return _timeseries_of

def omit_outliers(series, sigma=2, window=5):
    mavg = [np.average(w) for w in sliding_window(window, series)]
    std = np.std(mavg)
    return np.array([x if np.abs(x) < sigma*std else np.nan for x in mavg])

def avg_finite(arr):
    return np.average(arr[np.isfinite(arr)])

def min_series(x):
    return omit_outliers(timeseries_of("avg_sell")(x), sigma=0.5, window=10)

## Generate Plots

In [13]:
import plotly.express as px
for yvalue in ["num", "min", "p50", "d_num", "d_min", "d_p50"]:
    df = pd.DataFrame(timeseries)
    df = df.sort_values(by="timestamp")

    if yvalue.startswith("d_"):
        dfg = df.pivot(index="timestamp", columns="item_name", values=yvalue[2:])
        dfg = dfg.diff().div(dfg.index.to_series().diff(), axis=0)
    else:
        dfg = df.pivot(index="timestamp", columns="item_name", values=yvalue)

    fig = px.scatter(dfg, title=yvalue, log_y=False)
    fig.write_html(f"{yvalue}.html")

# TukUI Export Strings

In [14]:
# drpannkoeken
"Tukui:Profile:fAvtmTX1w4X2KeGuuaCHONAKIvF8GkAuk2qOrvpfpmyddVIdw2MKIBB6CT9fMjEyUtUZydjTsbHQQER9I(ZoKRu30nvSRRyek77uw0vvkOOSPRAfsvQn9bZ7(ZmdqsKQxyF99(DoNVZ58DpZez4w3cG1av0HwF(UZzyzd01b2AiJbLYGmGks50SQoSz8snQ3ql76MqSg0OkucGtMQq28VSds5EwPCOMW)7qhWoCQQu)roy8YBlTqPslKRWCZkxAZfNlpadnSD7w25gB8VfiFKf7mYxUfzH5amtNbvTHvjaEfO9myWQWYv4oW98U9i7ul9rBXmYyJOoccro4yB8rpD2BwkBbc6lqqReNcs2r7T8OlW50m0wfywEZslKNXicU(KD6u4EmQSJ3r)ycbHOMVkZTZdx2odWgucUUTeA9YT5mz(SZqTlUS4R5DiNnxs46e7eCptz34SJs75nYoEEEYIAp(z0qB2hZL5H2H1MYfcOWaYoZVr37tDLZWE)cXoVNWGx0gWlWLL8J6fRrYRL2pcX5hYXKtRkgLdA0OS7fyPT4vf(erAAloSqFjOXUxgWtwudGEpZOSm35ocri)2H5GNa6PmyaopFqIZUbZG3xrGMxCompcfG7Iusklw3jcV7Q0RWJj5ZZPlMOSB3eGezW6FbR(il2rhVonXDp3X1WJUfVgwFJOjcB2snwEzziOgex2TpE)dNqG3N)NEhswi8Cblzz3E4Yh7VlY(CnxSV)YexB2)js3Pbw20sDqQ2WjcpvRK(VowRf61ubEvu8H)bxg0z0AohRKlOTI6P0pbPCvNouy4Nr4k0f(SiVo4(qCil87qccR7Wyr50)pAfDBol0HyBRYPi6ycSHKfJF4xX8PJ6hDiDXM8AfyLqh1eDucgI525pPiAZdkaBF)YyIF8fyJilEDp(LmN7Ki6Jd1IZIrnmPn6YUJWXT2p4XJzDhHToopkHq62AMtzuvfHdlsX9K4fPmc9q5rlEncOvluZCjHo5AMmBiqR87T4CzXye2)QT7q8uLOu3N1THIXOYI9cUzDkPTJDIZXff31mgJD8BGzGvEjcOXIXUX4K7H)2gHf5BRvJilcIRt90hLMkhDkM2BRxuLDTaHdrZUedO4veIU)X6wKfprsgQWSf8VKONNva3oSMmXyHv53qOtpwyh373fdVXvsfdbCHIp9A(Rm9HSZB69ZekCK)T(PiZBbbX1pDThZ)w)DtZMl6dLlapDv0eDeVh)b78mcfJ1kqqB1KGKm)VM1OkNR10QamOQnepQYzkmBdn9AJQetAwe93EQu8(w2WvhvPlPsARchv5cTchAnQY5pGySn7YISwTAqdfj6gd2kpYsJwBTAwHXPXBI5AQKUJ0)7fP9CwlAudvToSMQ)bPChPImK6PN2z)lfPaPbARo0L4Mprt3PCJtm0n7J63DAIpwQl3mYIl65rNj4kPWt9KjPaPrWvsLIZndZBUzd801(7am5FhGuVaazXp8HmE0)sYoxxqGw0pgFYqUFsh2ffq7qbOv3BwI0eQRzSYWPY3mF3KVY2TzV3gH1RLdygupd7Td90VM0sJklo0t)yIYk6MfH22eRT(g3oU1tcmlRb9vfun7DHk3fsIvtilkqSQ5az0SmjIg(fL8O1GycpuBpvdBubOjqdRUlDDrOU()PHrDv8SqdigOp4U(pjUyvGo8tFf35)jZEVjcVkqxQHTnYOO2dGxRImY(DH3xXSp6qC(bw5H4cO1(hMXj7rUZFQDJTNV1MGQK8iMzFfxtZUQkb6cg8NLQ2UOkWewuvBzBvMJh)0oEtMsKSppSt8CHLUxQtVhM(Ecud2lKpm6lrM(Anyf)6N7nMSnZ1mr5J6VY8OvYbwpZL3Ml9zY2ADvGnqEy3vuEImzhD6UtJ0r43js(Zm2vh7TtgR9IgA8zDwFwB2R(mvdmWsvIoHrDVfn01mQ7)yefYB6uIF828R30hFArAr0Xc8dArTJVSaBBvj6iOjD19J5eJhRnNF08QElwgqx(Tse9H1W753mNgHW1SuoW)VbIrLcmNp52SqZlA)RDNwhyzPvLoqaR0AkQfuXPY))p"
# trowel
"Tukui:Profile:Tz1smTX1A4X2KecPOa4crv3ibQLYfHIsbdHg1wfpmyddn4GLTjT4EB7CSNdmdEyotpZyEKgPyHQQ6AVOp2HCL6MUPID3vmc19DUS4UnOOSPDvfsTQnTW0ZJXJJtIxyF8589)67)7)mtKrQFxawhuYaA)LhUGPTdWWa4OJmhqkfYeQiLr3U8iwXluTsv90BBbX6qZYqjaEIe5sN9fDqcVZlLbTj8lg(e2HZuM6pYbtwCFPLkuyPm5wyE5c7U8czbyOPJxxYU3Q27iq(il2zKVEpYcR(zMohQCv7ca8AqN5WGnGflXDG3L86w2vn5z7XmYSwuxbHiN0YMa0ZM(ofsNJG(Ye0kXPGKD1FdF6cCgDt9nawf3TWszzzebxVYUDk8jSu5a)Z(Fdjie16LzUDr4QoPaoGcWTDKqBxSbptwm9Cu7Ill(V8pLNnxv4Me7e8oxrV4SJs67)VpW33xwu)HpHgAREzUml0jKBkMRzk0VS7I166yQRChX)Nj25)ig88oaobxukiQxrLuxRCCeIZpLJjJEzmkd0SArVlZkBXRlCFrAzloIqVdrJDpmGpnP2e6NyfLv5UFKqeYVDynWtbTnd6NNN3BOZxJzWhOiqRlEoSic1e3vOjPSyf3i8URspcpKuppJUyQIEDrasKbB)vm(rwSJoEvAH7DHwC4z3LZHvQfDOWMTu1vxvgcuH4IE9Y7F4He495xZ)uYcHNjytu0RBU8X5hICmxZf7)oiX1w99uL7SaBhkv3SuR6gHxQLs(xT0AHEnrtVkk(G)Gld6mQQBlLCo910At)0SKl72Hcd)CcxJUiilYAa2bIdZIGoKGW2USSOyY)MYO7ZZcdi2XUycIoMaByzX4N(nmF6Q9XNsxSlNRaRf6OnrNnedXch8Nuen4bfGD2jSR92((jzmU6G(KApkV7opgv1I2IlIjXlqioQS4w)KppMvCf2RvDuaHmC0TMXSSgchssX9L4KukHUP5rDohb0vdJ(vf6KRzsvtGY8hT8cPXyeoy02ByEPsuQhZ62qXyuzXrnNSAtA76m0f4II1TIXYo(eykyPxGaA8ySjg3mp4xRfsYVNUkrw0mUUvsEgJCCZN0FVNxLDJMchIMDfolEnHOh3s3IS5fYeHkmhHGHeJSmcC)qozQXl6nkNLhvOtFwyN0)3edN4kOHHaUqji9wFWF)HmCt4)l1An1pd5(wqZ4guUoJhm1VEs29Ibq5cW2zrl0z8E8)5GNqsXy1BkOT3KGKC)VQ9ykxO(SAamOSdepMY5YnFvDd1XuIjnpI(B3LYVJTdCJXuUOub9nGJPC56HxAnMYLoHySdByrwxvfAQir3yG6zr26uU1E1sSCAYnXCn1eEJ237hPXc2lBQIkxbQQfCqcVrljdPE6XD23krYrAGoAdFvU5tTP3mEXjg6jP4nlHDtrbDrV0YI3NP2f44Ubf3OmCATWfXlDthnDiaYwue0tjGi45as8CEqw8dFGp9MN(wr29Mcm1DlhorRmtRTmB4R2iuEz31UfiuCfDZ1gjr2nZ2f5R0Dz1Z7HWgQzawnzRWo3Wp(BjnSOYId)4pLOBIUBEOJdXA7VZRJ7(OMML2K(IaAw9SuP1HKyTjKffiwZQ)u62wejbFmilAliMKhAnMPQdkh0cOJ1oKUop0W4DRAwrdpp0eIbgdCyWZzZxgya)SxY72)FREUdcVbWqQQJdYmV(9G3OKmY52WDuS6LEfn)a7SqCo0wVIvCYEKj622n2rbwBbktQJyw9MFlDNYAeOlzYFsPwJ8AalyEn9vD0yoEY2D8UmDgzFEyN6zclDVeTVhM(waudokmFyPVe5Uv7bkfWFE3A6gmxZKC)yFLweTwgW2PgCFUWMjkvVyo21TJ4TQYJKj7yq3DwKbc)wrYEUXV(4V5eXASSPo)Mm7pVb7fBMPkgyRjrV)q7OLnn0nRe8qcfY7XuGF8(8Hx6dhTjTi6qp)G6u74lZX2wtIEbZ0EvcI5utgRbp)O1vL6SkGU87Li6d7rokOzolcHvTvoj4VnfJk5yoF69zHMtAV(HZAaST1lth3Xk1NHAbvCQ8pd"

'Tukui:Profile:Tz1smTX1A4X2KecPOa4crv3ibQLYfHIsbdHg1wfpmyddn4GLTjT4EB7CSNdmdEyotpZyEKgPyHQQ6AVOp2HCL6MUPID3vmc19DUS4UnOOSPDvfsTQnTW0ZJXJJtIxyF8589)67)7)mtKrQFxawhuYaA)LhUGPTdWWa4OJmhqkfYeQiLr3U8iwXluTsv90BBbX6qZYqjaEIe5sN9fDqcVZlLbTj8lg(e2HZuM6pYbtwCFPLkuyPm5wyE5c7U8czbyOPJxxYU3Q27iq(il2zKVEpYcR(zMohQCv7ca8AqN5WGnGflXDG3L86w2vn5z7XmYSwuxbHiN0YMa0ZM(ofsNJG(Ye0kXPGKD1FdF6cCgDt9nawf3TWszzzebxVYUDk8jSu5a)Z(Fdjie16LzUDr4QoPaoGcWTDKqBxSbptwm9Cu7Ill(V8pLNnxv4Me7e8oxrV4SJs67)VpW33xwu)HpHgAREzUml0jKBkMRzk0VS7I166yQRChX)Nj25)ig88oaobxukiQxrLuxRCCeIZpLJjJEzmkd0SArVlZkBXRlCFrAzloIqVdrJDpmGpnP2e6NyfLv5UFKqeYVDynWtbTnd6NNN3BOZxJzWhOiqRlEoSic1e3vOjPSyf3i8URspcpKuppJUyQIEDrasKbB)vm(rwSJoEvAH7DHwC4z3LZHvQfDOWMTu1vxvgcuH4IE9Y7F4He495xZ)uYcHNjytu0RBU8X5hICmxZf7)oiX1w99uL7SaBhkv3SuR6gHxQLs(xT0AHEnrtVkk(G)Gld6mQQBlLCo910At)0SKl72Hcd)CcxJUiilYAa2bIdZIGoKGW2USSOyY)MYO7ZZcdi2XUycIoMaByzX4N(nmF6Q9XNsxSlNRaRf6OnrNnedXch8Nuen4bfGD2jSR92((jzmU6G(KApkV7opgv1I2IlIjXlqioQS4w)KppMvCf2RvDuaHmC0TMXSSgchssX9L4KukHUP5rDo

# BIS Filtering from Loon Data

In [15]:
import re
import pandas as pd

loon_path = "C:/Program Files (x86)/World of Warcraft/_classic_/Interface/AddOns/LoonBestInSlot"

item_sources_path = f"{loon_path}/DB/ItemSources.lua"

item_sources_index = {}

item_pickups = []

with open(item_sources_path) as f:
    for entry in f:
        if not entry.strip():
            continue
        try:
            item_id = int(re.search(r"^\s+\[(\d+)\]", entry).group(1))
        except AttributeError:
            continue
        kvs = re.search(r'[A-Z].+ = .*?(?:,| \})', entry).group(0)
        kvs1 = re.sub(r"LBIS.L\[(.*?)\]", r"\1", kvs).replace(r'".."', "")
        kvs2 = re.sub(r"\s+\}$", "", kvs1)
        kvs3 = re.split(r",\s+", kvs2)
        kvs4 = [re.split(r" = ", kv) for kv in kvs3]

        d = {}
        acc = None
        for kv in kvs4:
            # Normal case
            if len(kv) == 2:
                d[kv[0]] = kv[1].strip('"')
            # There was a ' = ' in the name
            elif len(kv) > 2:
                d[kv[0]] = " = ".join(k.strip('"') for k in kv[1:])
            # Hmmmm, shouldn't happen but not harmful if it does
            elif len(kv) == 0:
                continue
            # There was a ', ' in the name; we need to start a join state
            elif len(kv) == 2 and kv[1].count('"') == 1 and kv[1].startswith('"'):
                acc = (kv[0], kv[1].strip('"'))
            # Join state
            elif acc and len(kv) == 1 and kv[0].count('"') == 0:
                acc[1] = acc[1] + ", " + kv[0]
            # End the join state
            elif acc and len(kv) == 1 and kv[0].count('"') == 1:
                acc[1] = acc[1] + ", " + kv[1].strip('"')
                d[acc[0]] = acc[1].strip('"')
                acc = None

        source_split = {k: v.split("/") for (k, v) in d.items() if k != "Name"}

        split_entries = [
            {
                "Id": item_id,
                "Name": d["Name"],
                **dict(zip(source_split.keys(), vs))
            }
            for vs in zip(*list(source_split.values()))
        ]
        item_pickups.extend(split_entries)
        if item_id in item_sources_index:
            item_sources_index[item_id].extend(split_entries)
        else:
            item_sources_index[item_id] = split_entries

            
ranks = [
    "Alt Mit",
    "Alt Stam",
    "Alt Thrt",
    "Alt Transmute",
    "Alt",
    "Alt/BIS",
    "BIS/Alt",
    "BIS Mit",
    "BIS Stam",
    "BIS Thrt",
    "BIS",
]
rank_order = {r: i for (i, r) in enumerate(reversed(ranks))}

def sortable_rank(phase, rank):
    return -int(phase)*len(ranks) + rank_order[rank]

item_guide_path = f"{loon_path}/Guides"
guide_files = glob.glob(f"{item_guide_path}/Phase*/*.lua")

item_guide_index = {}

guide_records = []

for file_ in guide_files:
    print(f"{file_=}")
    with open(file_) as f:
        meta = {}
        for line in f:
            if not line.strip():
                continue
            print(line.strip())
            if "RegisterSpec" in line:
                m = re.search(
                    (
                        r'RegisterSpec\(LBIS.L\["(?P<class>.*?)"\], '
                        r'LBIS.L\["(?P<spec>.*?)"\], '
                        r'"(?P<phase>\d+)"\)'
                    ),
                    line,
                )
                meta = m.groupdict()
            elif meta and line:
                (item_id_str, slot, rank) = [x.strip('"') for x in re.findall(r'".*?"', line)]
                item_id = int(item_id_str)
                record = {
                    "Id": item_id,
                    "Name": items.get_name(item_id),
                    "Slot": slot,
                    "Rank": rank,
                    "Phase": meta["phase"],
                    "Class": meta["class"],
                    "Spec": meta["spec"],
                    "ReadableRank": f"P{meta['phase']} {rank}",
                    "SortableRank": sortable_rank(meta['phase'], rank),
                }
                guide_records.append(record)
                if item_id in item_guide_index:
                    item_guide_index[item_id].append(record)
                else:
                    item_guide_index[item_id] = [record]
            elif line:
                raise ValueError("File metadata did not appear before AddItem entries")


bis_raw = []
for (item_id, records) in item_guide_index.items():
    sources = item_sources_index[item_id]
    for record in records:
        for source in sources:
            bis_raw.append({**record, **source})


bis = pd.DataFrame(bis_raw)

file_='C:/Program Files (x86)/World of Warcraft/_classic_/Interface/AddOns/LoonBestInSlot/Guides\\Phase0\\DeathKnightBlood.lua'
local spec = LBIS:RegisterSpec(LBIS.L["Death Knight"], LBIS.L["Blood"], "0")
LBIS:AddItem(spec, "39625", LBIS.L["Head"], "BIS") --Heroes' Scourgeborne Faceguard
LBIS:AddItem(spec, "39627", LBIS.L["Shoulder"], "BIS") --Heroes' Scourgeborne Pauldrons
LBIS:AddItem(spec, "43988", LBIS.L["Back"], "BIS") --Gale-Proof Cloak
LBIS:AddItem(spec, "39398", LBIS.L["Chest"], "BIS") --Massive Skeletal Ribcage
LBIS:AddItem(spec, "40734", LBIS.L["Wrist"], "BIS") --Bracers of Dalaran's Parapets
LBIS:AddItem(spec, "39624", LBIS.L["Hands"], "BIS") --Heroes' Scourgeborne Handguards
LBIS:AddItem(spec, "40689", LBIS.L["Waist"], "BIS") --Waistguard of Living Iron
LBIS:AddItem(spec, "39626", LBIS.L["Legs"], "BIS") --Heroes' Scourgeborne Legguards
LBIS:AddItem(spec, "40743", LBIS.L["Feet"], "BIS") --Kyzoc's Ground Stompers
LBIS:AddItem(spec, "40679", LBIS.L["Neck"], "BIS") --Chained Mi

In [16]:
import itertools
from functools import partial
from functools import reduce


def bis_filter(k1, k2):
    return bis[k1] == k2


def bis_or(entries):
    entries = iter(entries)
    start = next(entries)
    return reduce(lambda acc, x: acc | x, entries, start)


bfclass = partial(bis_filter, "Class")
bfspec = partial(bis_filter, "Spec")
bfloc = partial(bis_filter, "SourceLocation")
bfsrc = partial(bis_filter, "Source")
bfrank = partial(bis_filter, "Rank")
bfrerank = partial(bis_filter, "ReadableRank")
bfphase = partial(bis_filter, "Phase")
bfslot = partial(bis_filter, "Slot")
bftype = partial(bis_filter, "SourceType")


def raidnames(name):
    return [f"{name}", f"{name} (10)", f"{name} (25)"]

raids_ = [
    "Naxxramas",
    "The Obsidian Sanctum",
    "The Eye of Eternity",
    "Vault of Archavon",
    "Ulduar",
]

raids = list(itertools.chain.from_iterable(raidnames(n) for n in raids_))


def bis_display(filters, drop_columns=None, show_args=None):
    drop_columns = drop_columns or []
    show_args = show_args or {}
    return show(without_columns(drop_columns, bis[filters]), **show_args)

In [ ]:
purchases = summarize_tsm_by_mask((df_all["mode"] == "purchases") & (df_all["source"] != "Vendor"))
purchases["volatility pct"] = 100 * purchases["price volatility"] / purchases["unit price"]
purchases.sort_values("volatility pct", ascending=False)

In [70]:
too_hard = [
    "Mimiron (Hard)",
    "Algalon the Observer (Hard)",
    "Algalon the Observer & Mimiron (Hard)",
    "General Vezax (Hard)",
    "Yogg-Saron (Hard)",
]
bis_display(
    (
        bfclass("Warlock") &
        bfspec("Affliction") &
        ~bis_or(bfsrc(x) for x in too_hard) &
        bis_or(bfrerank(x) for x in ["P2 Alt", "P2 BIS"])
    ),
    drop_columns=["Id", "SourceNumber", "SourceType", "Class", "Spec", "Phase", "Rank",],
    show_args={"lengthMenu": [20, 10, 30, 50, 100], "order": [[4, "asc"]]},
)

In [64]:
z = [210, 750, 802, 925, 1333, 5387]
[
    [zz/y for zz in z]
    for y in z
]

[[1.0,
  3.5714285714285716,
  3.8190476190476192,
  4.404761904761905,
  6.347619047619047,
  25.652380952380952],
 [0.28,
  1.0,
  1.0693333333333332,
  1.2333333333333334,
  1.7773333333333334,
  7.182666666666667],
 [0.26184538653366585,
  0.9351620947630923,
  1.0,
  1.153366583541147,
  1.6620947630922693,
  6.716957605985037],
 [0.22702702702702704,
  0.8108108108108109,
  0.867027027027027,
  1.0,
  1.441081081081081,
  5.823783783783783],
 [0.15753938484621155,
  0.5626406601650412,
  0.6016504126031508,
  0.6939234808702176,
  1.0,
  4.04126031507877],
 [0.03898273621681827,
  0.1392240579172081,
  0.1488769259328012,
  0.17170967143122332,
  0.24744755893818451,
  1.0]]

In [66]:
from cytoolz import sliding_window
[y/x for (x, y) in sliding_window(2, z)]

[3.5714285714285716,
 1.0693333333333332,
 1.153366583541147,
 1.441081081081081,
 4.04126031507877]

In [67]:
4*(851+148+119)

4472